The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash

#!/bin/bash

cd $HOME

# a)
# make students directory
mkdir -p $HOME/students
# download file from URL
# -O allowes to change the filename directly
# be careful putting & at the end of the url that means to download in background ––> bugs on the flow of the script
wget -O LCP_students.csv https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7
# One can also use wget and than mv

# check wether the file already is inside the directory, if not copy inside of it
if [ -f "./students/LCP_students.csv" ]
then
    echo "The file ./students/LCP_students.csv already exists."
else 
    cp LCP_students.csv ./students/
fi

# b)
# create 2 new files in the students directory
touch ./students/PoD_stud.csv
touch ./students/Phys_stud.csv

# c,d)
# case-insensitive search 
grep -i "physics" ./students/LCP_students.csv > ./students/Phys_stud.csv 
grep -i "pod" ./students/LCP_students.csv > ./students/PoD_stud.csv 

cd students

max_count_PoD=0
letter_PoD=""
for i in {A..Z}; do
    n=$( grep -c "^$i" PoD_stud.csv ) # -c counts the number of lines matching the requirement (also displays the result)
    echo "Number of lines starting with $i in PoD: $n"
    if [ "$n" -gt "$max_count_PoD" ]; then 
        max_count_PoD=$n
        letter_PoD=$i
    fi
done

echo "\n"

max_count_Phys=0
letter_Phys=""
for i in {A..Z}; do
    n=$( grep -c "^$i" Phys_stud.csv )
    echo "Number of lines starting with $i in Physics: $n"
    if [ "$n" -gt "$max_count_Phys" ]; then 
        max_count_Phys=$n
        letter_Phys=$i
    fi
done

echo "The letter with most counts in PoD is: $letter_PoD"
echo "The letter with most counts in Phys is: $letter_Phys"

# e)
counter=1
while IFS= read -r line; do # reads the file line by line without trimming
    if [ $counter -eq 1 ]; then 
        ((counter++))
        continue 
    fi
    group=$(( (counter - 1) % 18 + 1 ))
    echo "$line" >> "group_$group.txt"
    ((counter++))
done < LCP_students.csv



```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash
#!/bin/bash

cd $HOME/anaconda_projects/LaboratoryOfPhysicsOfData_Y8

# a) Copyng data payload in a file data.txt
# "grep -v "^#" data.csv | cut -f6 -d "," > data.txt" –> mi mette in colonna un numero per volta
grep -v "^#" data.csv | sed 's/,/\t/g' > data.txt

#b)
even_counter=0
while IFS=$'\t' read -r element; do # reads the file line by line separating line element with \t (but takes the whole line)
    for num in $element; do
        (( $num % 2 == 0 )) && (( even_counter++ ))
    done
done < data.txt
echo "The count of even numbers is $even_counter"

#c)
group1=0
group2=0
comp=$( echo "scale=3; 100*sqrt(3)/2" | bc -l )
while IFS=$'\t' read -r X Y Z _; do # "_" ignores the rest of the line (also cut ... can be used at the end)
    d=$( echo "scale=3; sqrt($X^2 + $Y^2 + $Z^2)" | bc -l )
    if (( $(echo "$d > $comp" | bc -l) )); then
        ((group1++))
    else
        ((group2++))
    fi
done < data.txt
echo "Number of elements in group 1: $group1"
echo "Number of elements in group 2: $group2"

#d)
if [ -z "$1" ]; then
    echo "This script requires as input a natural number"
    exit 1 # error code
fi

mkdir -p data_div

for (( i=1; i<=$1; i++ )); do

    while IFS=$'\t' read -r X Y Z Xp Yp Zp; do # reads the file line by line separating line element with \t
        # risultato=$( echo "scale=3; $num / $i" | bc )
        # echo "$risultato" >> data_div/$i.txt –––> inserisce i risultati in una colonna
        echo -e "$(echo "scale=3; $X/$i" | bc)\t$(echo "scale=3; $Y/$i" | bc)\t$(echo "scale=3; $Z/$i" | bc)\t$(echo "scale=3; $Xp/$i" | bc)\t$(echo "scale=3; $Yp/$i" | bc)\t$(echo "scale=3; $Zp/$i" | bc)\t" >> data_div/"$i.txt"
    done < data.txt
    
done

```